### 1. Predecir el valor del dolar utilizando una red neuronal

Resolver el problema de predecir el valor del dólar, que se planteó en el capítulo
anterior, utilizando una red neuronal. Describir las diferencias entre un modelo de
entrenamiento y el otro

In [2]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

def limpiar_y_convertir_a_float(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].str.replace('.', '').str.replace(',', '.').astype(float)
    return df

# Cargar los datos
bancolombia_df = pd.read_csv('Datos históricos BIC (1).csv')
ecopetrol_df = pd.read_csv('Datos históricos ECO (2).csv')
petroleo_df = pd.read_csv('Datos históricos Futuros petróleo Brent.csv')
tasa_cambio_df = pd.read_csv('Tasa_de_Cambio_Representativa_del__Mercado_-Historico_20240404 (1).csv')

columnas_para_convertir = ['Último', 'Apertura', 'Máximo', 'Mínimo'] 

bancolombia_df = limpiar_y_convertir_a_float(bancolombia_df, columnas_para_convertir)
ecopetrol_df = limpiar_y_convertir_a_float(ecopetrol_df, columnas_para_convertir)
petroleo_df = limpiar_y_convertir_a_float(petroleo_df, columnas_para_convertir)

# Asegurándonos de que las fechas estén en formato uniforme
bancolombia_df['Fecha'] = pd.to_datetime(bancolombia_df['Fecha'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
ecopetrol_df['Fecha'] = pd.to_datetime(ecopetrol_df['Fecha'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
petroleo_df['Fecha'] = pd.to_datetime(petroleo_df['Fecha'], format='%d.%m.%Y').dt.strftime('%Y-%m-%d')
tasa_cambio_df['Fecha'] = pd.to_datetime(tasa_cambio_df['VIGENCIADESDE'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')

bancolombia_precios = bancolombia_df[['Fecha', 'Último']].rename(columns={'Último': 'Precio_Bancolombia'})
ecopetrol_precios = ecopetrol_df[['Fecha', 'Último']].rename(columns={'Último': 'Precio_Ecopetrol'})
petroleo_precios = petroleo_df[['Fecha', 'Último']].rename(columns={'Último': 'Precio_Petroleo'})
tasa_cambio = tasa_cambio_df[['Fecha', 'VALOR']].rename(columns={'VALOR': 'Tasa_Cambio'})

# Combinando los dataframes
datos_combinados = pd.merge(bancolombia_precios, ecopetrol_precios, on='Fecha', how='outer')
datos_combinados = pd.merge(datos_combinados, petroleo_precios, on='Fecha', how='outer')
datos_combinados = pd.merge(datos_combinados, tasa_cambio, on='Fecha', how='outer')

# Ordenando los datos por fecha
datos_combinados = datos_combinados.sort_values('Fecha')



# Interpolación y eliminación de filas con valores faltantes
datos_completos = datos_combinados.interpolate().dropna()

# Definir las variables X e y
X = datos_completos[['Precio_Bancolombia', 'Precio_Ecopetrol', 'Precio_Petroleo']]
y = datos_completos['Tasa_Cambio']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de red neuronal
nn_model = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', random_state=42, max_iter=500))
nn_model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred = nn_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f'MSE: {mse}, R^2: {r2}, RMSE: {rmse}')


C:\Users\dmriv\AppData\Local\Temp\ipykernel_2984\179312154.py:48: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  datos_completos = datos_combinados.interpolate().dropna()


MSE: 76677.19990005113, R^2: 0.8954941716314369, RMSE: 276.9064822282987


c:\Users\dmriv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MSE (Error Cuadrático Medio): 76,677.20. Un valor más bajo indica un mejor ajuste.

R² (Coeficiente de Determinación): 0.895. Esta es una métrica clave que indica cuánta variabilidad de tus datos puede ser explicada por el modelo.valor de 0.895 sugiere que el modelo explica aproximadamente el 89.5% de la variabilidad, lo cual es bastante bueno.

RMSE (Raíz del Error Cuadrático Medio): 276.91. Esta métrica ofrece una medida de la magnitud del error, proporcionando una idea de cuán grandes son los errores en términos de las unidades originales de los datos